# Q3: Ridge Regression

Introduction to Data Science (DS-GA1001)


#### Import libraries

In [29]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, LeaveOneOut, LeavePOut, validation_curve, learning_curve, GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import Lasso, Ridge, ElasticNet, RidgeCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold


import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

#### Ingest data

In [30]:
df = pd.read_csv('movieReplicationSet.csv')


In [31]:
df

,The Life of David Gale (2003),Wing Commander (1999),Django Unchained (2012),Alien (1979),Indiana Jones and the Last Crusade (1989),Snatch (2000),Rambo: First Blood Part II (1985),Fargo (1996),Let the Right One In (2008),Black Swan (2010),...,When watching a movie I cheer or shout or talk or curse at the screen,When watching a movie I feel like the things on the screen are happening to me,As a movie unfolds I start to have problems keeping track of events that happened earlier,"The emotions on the screen ""rub off"" on me - for instance if something sad is happening I get sad or if something frightening is happening I get scared",When watching a movie I get completely immersed in the alternative reality of the film,Movies change my position on social economic or political issues,When watching movies things get so intense that I have to stop watching,Gender identity (1 = female; 2 = male; 3 = self-described),Are you an only child? (1: Yes; 0: No; -1: Did not respond),Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)
0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,1.0,6.0,2.0,5.0,5.0,5.0,1.0,1.0,0,1
1,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,1.0,6.0,5.0,3.0,2.0,1.0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,4.0,3.0,5.0,5.0,4.0,4.0,1.0,1,0
3,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,4.0,...,3.0,1.0,1.0,4.0,5.0,3.0,1.0,1.0,0,1
4,NaN,NaN,3.5,NaN,0.5,NaN,0.5,1.0,NaN,0.0,...,2.0,3.0,2.0,5.0,6.0,4.0,4.0,1.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,...,3.0,4.0,3.0,5.0,5.0,4.0,4.0,1.0,0,0
1093,3.0,4.0,NaN,NaN,4.0,4.0,2.5,NaN,3.5,3.5,...,5.0,3.0,5.0,5.0,5.0,6.0,5.0,1.0,0,0
1094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,...,6.0,3.0,1.0,6.0,6.0,4.0,2.0,1.0,0,0
1095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,4.0,3.0,3.0,1.0,1.0,0,1


In [32]:
print(len(df.columns))     # 477 without 'mean', 478 with 'mean'

477


In [33]:
# creates a dictionary of the mean value of each column

# returns a dictionary of column and its mean value
def make_column_means(M):
    column_means = {}
    
    for col in df.columns:
#         print(col)
        colmean = df[col].mean()
        column_means[str(col)] = colmean

    return column_means

In [34]:
def append_rowmeans(M):

    M['mean'] = M.mean(axis=1)
#     return M

In [35]:
# replace nans 
# first gets all of the column means
# then makes a new column for the rowmean
# for a given nan, replace it with the average of its rowmean and its colmean
def enrich_nans(M):
    
    column_means = make_column_means(M)
#     M = append_rowmeans(M)
    append_rowmeans(M)
    
    # for every column
    for col in M.columns:

        for row in range(len(M)):
            value = M.at[row, col]

            if np.isnan(value):
                rowmean = M.at[row, 'mean']
                colmean = column_means[col]
                newvalue = (rowmean + colmean) / 2

#                 print("rowmean: %s, colmean: %s ==> NEW: %s" % (rowmean, colmean, newvalue))
                M.at[row, col] = newvalue
                
    return M

In [36]:
cols = df.columns.tolist()
b_cols = cols[400:]
rating_cols = cols[:400]

In [37]:
# get a dataframe of the movies only
df = df[rating_cols]

In [38]:
df.isna().sum().sum()

326586

In [40]:
# add a placeholder for the new column
df['mean'] = np.nan

/var/folders/c7/k2fcvfxj5610zmlmq4sn39z00000gn/T/ipykernel_13915/4117475601.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mean'] = np.nan


In [41]:
# do the missing value handling as described in the doc
df = enrich_nans(df)
df

/var/folders/c7/k2fcvfxj5610zmlmq4sn39z00000gn/T/ipykernel_13915/1662261430.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  M['mean'] = M.mean(axis=1)


,The Life of David Gale (2003),Wing Commander (1999),Django Unchained (2012),Alien (1979),Indiana Jones and the Last Crusade (1989),Snatch (2000),Rambo: First Blood Part II (1985),Fargo (1996),Let the Right One In (2008),Black Swan (2010),...,The Usual Suspects (1995),The Mask (1994),Jaws (1975),Harry Potter and the Chamber of Secrets (2002),Patton (1970),Anaconda (1997),Twister (1996),MacArthur (1977),Look Who's Talking (1989),mean
0,2.447086,2.381992,4.000000,2.725235,3.000000,2.670257,2.554121,2.821232,2.619604,2.827211,...,2.921947,2.650951,4.000000,0.500000,2.510773,2.519156,2.572578,2.428806,2.540410,2.742857
1,2.439294,2.374200,1.500000,2.717443,2.752945,2.662464,2.546329,2.813440,2.611812,2.819419,...,2.914154,2.643159,2.673112,4.000000,2.502981,2.511364,2.564786,2.421013,2.532618,2.727273
2,2.733065,2.667971,3.234118,3.011214,3.046716,2.956236,2.840100,3.107211,2.905583,3.113190,...,3.207926,2.936930,2.966883,3.500000,2.796752,2.805135,2.858557,2.714784,2.826389,3.314815
3,2.282975,2.217880,2.000000,2.561123,3.000000,2.506145,2.390009,2.657120,2.455492,4.000000,...,3.000000,2.486840,2.516793,2.500000,2.346661,2.355044,2.408466,2.264694,2.376299,2.414634
4,2.209132,2.144038,3.500000,2.487281,0.500000,2.432303,0.500000,1.000000,2.381650,0.000000,...,2.683993,3.000000,2.442950,2.769704,2.272819,2.281202,1.500000,2.190852,2.302456,2.266949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,2.675658,2.610563,3.176711,2.953806,3.500000,2.898828,2.782692,3.049803,2.848175,3.055782,...,3.150518,2.879523,2.909476,4.000000,2.739344,2.747727,2.801149,2.657377,2.768981,3.200000
1093,3.000000,4.000000,3.413546,3.190641,4.000000,4.000000,2.500000,3.286638,3.500000,3.500000,...,3.387353,4.000000,3.500000,3.500000,4.000000,3.500000,4.000000,4.000000,4.000000,3.673669
1094,2.641923,2.576828,3.142976,2.920071,2.955574,2.865093,2.748957,3.500000,2.814440,3.022047,...,3.116783,2.845788,2.875741,4.000000,2.705609,2.713992,2.767414,2.623642,2.735247,3.132530
1095,2.770970,2.705876,3.272023,3.049119,3.084621,2.994141,2.878005,3.145116,2.943488,3.151095,...,3.245831,2.974835,3.004788,2.500000,2.834657,2.843040,2.896462,2.752690,2.864294,3.390625


In [42]:
df.isna().sum()

The Life of David Gale (2003)                1
Wing Commander (1999)                        1
Django Unchained (2012)                      1
Alien (1979)                                 1
Indiana Jones and the Last Crusade (1989)    1
                                            ..
Anaconda (1997)                              1
Twister (1996)                               1
MacArthur (1977)                             1
Look Who's Talking (1989)                    1
mean                                         1
Length: 401, dtype: int64

In [43]:
df = df.dropna()

In [44]:
df.isna().sum()

The Life of David Gale (2003)                0
Wing Commander (1999)                        0
Django Unchained (2012)                      0
Alien (1979)                                 0
Indiana Jones and the Last Crusade (1989)    0
                                            ..
Anaconda (1997)                              0
Twister (1996)                               0
MacArthur (1977)                             0
Look Who's Talking (1989)                    0
mean                                         0
Length: 401, dtype: int64

In [45]:
df

,The Life of David Gale (2003),Wing Commander (1999),Django Unchained (2012),Alien (1979),Indiana Jones and the Last Crusade (1989),Snatch (2000),Rambo: First Blood Part II (1985),Fargo (1996),Let the Right One In (2008),Black Swan (2010),...,The Usual Suspects (1995),The Mask (1994),Jaws (1975),Harry Potter and the Chamber of Secrets (2002),Patton (1970),Anaconda (1997),Twister (1996),MacArthur (1977),Look Who's Talking (1989),mean
0,2.447086,2.381992,4.000000,2.725235,3.000000,2.670257,2.554121,2.821232,2.619604,2.827211,...,2.921947,2.650951,4.000000,0.500000,2.510773,2.519156,2.572578,2.428806,2.540410,2.742857
1,2.439294,2.374200,1.500000,2.717443,2.752945,2.662464,2.546329,2.813440,2.611812,2.819419,...,2.914154,2.643159,2.673112,4.000000,2.502981,2.511364,2.564786,2.421013,2.532618,2.727273
2,2.733065,2.667971,3.234118,3.011214,3.046716,2.956236,2.840100,3.107211,2.905583,3.113190,...,3.207926,2.936930,2.966883,3.500000,2.796752,2.805135,2.858557,2.714784,2.826389,3.314815
3,2.282975,2.217880,2.000000,2.561123,3.000000,2.506145,2.390009,2.657120,2.455492,4.000000,...,3.000000,2.486840,2.516793,2.500000,2.346661,2.355044,2.408466,2.264694,2.376299,2.414634
4,2.209132,2.144038,3.500000,2.487281,0.500000,2.432303,0.500000,1.000000,2.381650,0.000000,...,2.683993,3.000000,2.442950,2.769704,2.272819,2.281202,1.500000,2.190852,2.302456,2.266949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,2.675658,2.610563,3.176711,2.953806,3.500000,2.898828,2.782692,3.049803,2.848175,3.055782,...,3.150518,2.879523,2.909476,4.000000,2.739344,2.747727,2.801149,2.657377,2.768981,3.200000
1093,3.000000,4.000000,3.413546,3.190641,4.000000,4.000000,2.500000,3.286638,3.500000,3.500000,...,3.387353,4.000000,3.500000,3.500000,4.000000,3.500000,4.000000,4.000000,4.000000,3.673669
1094,2.641923,2.576828,3.142976,2.920071,2.955574,2.865093,2.748957,3.500000,2.814440,3.022047,...,3.116783,2.845788,2.875741,4.000000,2.705609,2.713992,2.767414,2.623642,2.735247,3.132530
1095,2.770970,2.705876,3.272023,3.049119,3.084621,2.994141,2.878005,3.145116,2.943488,3.151095,...,3.245831,2.974835,3.004788,2.500000,2.834657,2.843040,2.896462,2.752690,2.864294,3.390625


## Done with enriching and cleaning, on to simple linear regression model

In [46]:
# confirm the first movie column and the last one
df.columns[0] # wing commander, we need to index at 0
df.columns[399] # Look Whos Talking, the final movie col
max_index = 399
# df.columns[400] # -> 'I enjoy driving fast'

In [47]:
'''
reg = LinearRegression().fit(x.reshape(-1,1), y)
                # x = test values (if we were doing cross validation)
y_hat = reg.predict(x.reshape(-1,1))

r2 = r2_score(y,y_hat)   
print('R^2:', r2.round(3))
rmse = np.sqrt(np.mean(np.sum((y-y_hat)**2)))
print('RMSE:', rmse.round(3))
'''
def linreg(x,y, name1,name2):
    reg = LinearRegression().fit(x.reshape(-1,1), y)
    y_hat = reg.predict(x.reshape(-1,1))
    # y = m*x + b
    
    optimal_beta = float(reg.coef_.round(3))
    offset = reg.intercept_.round(3)           # offset is the y-intercept
    r2 = r2_score(y,y_hat)
    rmse = np.sqrt(np.mean(np.sum((y-y_hat)**2)))
    
    return r2, rmse


In [48]:
'''Please include a histogram of these 400 COD values and a table with the 10 movies that are most
easily predicted from the ratings of a single other movie and the 10 movies that are hardest to predict
from the ratings of a single other movie (and their associated COD values, as well as which movie
ratings are the best predictor, so this table should have 3 columns).
'''

def Average(lst):
    return sum(lst) / len(lst)


def rank_scores(obj):
    n = 1
    r2 = []
    for o_movie, vals in obj.items():
        r2.append(vals['r2'])

    average_r2 = Average(r2)
    
    best_ten = r2.copy()
    best_ten = sorted(best_ten, reverse=True)
    best_ten = best_ten[:n]
    
    worst_ten = r2.copy()
    worst_ten = sorted(worst_ten)
    worst_ten = worst_ten[:n]
            
#     print(best_ten)
#     print(worst_ten)
    
    # return an array of the objects
    # best
    obj_b = obj.copy()
    best = []
    for score in best_ten:
        for m, v in obj_b.copy().items():

            if v['r2'] == score:
                this_movie_item = {}
                this_movie_item[m] = v
                best.append(this_movie_item)
                
                # also remove it from objs
                del obj_b[m]
                
    obj_w = obj.copy()
    worst = []
    for score in worst_ten:
        for m, v in obj_w.copy().items():
            
            if v['r2'] == score:
                this_movie_item = {}
                this_movie_item[m] = v
                worst.append(this_movie_item)
                
                # also remove it from objs
                del obj_w[m]
        
            
    return best, worst, average_r2

In [49]:
# Wing Commander (1999)
# Look Who's Talking (1989)

cols = df.columns
movie_columns = cols[:400].tolist()

average_CODs = []
best_CODs = []
best_data_all = []

for movie in movie_columns:
    print("Finding best predictor: '%s'" % movie)
    
    this_movie_scores = {}
    x = pd.to_numeric(df[movie],errors='coerce').values

    other_movies = movie_columns.copy()
    other_movies.remove(movie)
    
    for other_movie in other_movies:
        #print("\tOther movie: '%s'" % other_movie)
        y = pd.to_numeric(df[other_movie],errors='coerce').values
        r2, rmse = linreg(x,y, movie,other_movie)
        this_movie_scores[other_movie] = {"x":x, "y":y, "r2":r2, "rmse":rmse}
        
    # assess scores for the 399 models made for this single movie
    best, worst, avg_cod = rank_scores(this_movie_scores)
    y_movie = dict(best[0])
    best_name = list(y_movie.keys())[0]
    best_cod = y_movie[best_name]['r2']
    best_data = {'movie': movie, 'other_movie': best_name, 'cod': best_cod}
#     best_data['movie'] = movie
    best_data_all.append(best_data)

    # i need name of movie and best cod score
    # i need a histogram of the average COD of each movie model
    average_CODs.append(avg_cod)
    best_CODs.append(best_cod)


Finding best predictor: 'The Life of David Gale (2003)'
Finding best predictor: 'Wing Commander (1999)'
Finding best predictor: 'Django Unchained (2012)'
Finding best predictor: 'Alien (1979)'
Finding best predictor: 'Indiana Jones and the Last Crusade (1989)'
Finding best predictor: 'Snatch (2000)'
Finding best predictor: 'Rambo: First Blood Part II (1985)'
Finding best predictor: 'Fargo (1996)'
Finding best predictor: 'Let the Right One In (2008)'
Finding best predictor: 'Black Swan (2010)'
Finding best predictor: 'King Kong (1976)'
Finding best predictor: 'The Machinist (2004)'
Finding best predictor: 'A Nightmare on Elm Street (1984)'
Finding best predictor: 'Brazil (1985)'
Finding best predictor: 'The Fast and the Furious (2001)'
Finding best predictor: 'Change of Habit (1969)'
Finding best predictor: 'American Beauty (1999)'
Finding best predictor: 'Psycho (1960)'
Finding best predictor: 'Terminator 3: Rise of the Machines (2003)'
Finding best predictor: 'Night of the Living Dead

Finding best predictor: 'Room (2015)'
Finding best predictor: 'Scream (1996)'
Finding best predictor: 'The Evil Dead (1981)'
Finding best predictor: 'Gangs of New York (2002)'
Finding best predictor: 'Stand By Me (1986)'
Finding best predictor: 'The Vow (2012)'
Finding best predictor: 'Toy Story 3 (2010)'
Finding best predictor: 'The Matrix Reloaded (2003)'
Finding best predictor: 'Once Upon a Time in the West (1968)'
Finding best predictor: 'Star Wars: Episode V - The Empire Strikes Back (1980)'
Finding best predictor: 'War Games (1983)'
Finding best predictor: 'Kill Bill: Vol. 2 (2004)'
Finding best predictor: 'Saving Private Ryan (1998)'
Finding best predictor: 'Just Married (2003)'
Finding best predictor: 'Being John Malkovich (1999)'
Finding best predictor: 'Father's Day (1997)'
Finding best predictor: 'Batman (1989)'
Finding best predictor: 'Se7en (1995)'
Finding best predictor: 'Happy Gilmore (1996)'
Finding best predictor: 'My Big Fat Greek Wedding (2002)'
Finding best predicto

Finding best predictor: 'You're Next (2011)'
Finding best predictor: 'The Final Conflict (1981)'
Finding best predictor: 'City of God (2002)'
Finding best predictor: 'Star Wars: Episode VII - The Force Awakens (2015)'
Finding best predictor: 'The Transporter (2002)'
Finding best predictor: 'Cast Away (2000)'
Finding best predictor: 'Bad Boys 2 (2003)'
Finding best predictor: 'The Babadook (2014)'
Finding best predictor: 'Saw (2004)'
Finding best predictor: 'Star Wars: Episode VI - The Return of the Jedi (1983)'
Finding best predictor: 'Scary Movie (2000)'
Finding best predictor: 'E.T. The Extra-Terrestrial (1982)'
Finding best predictor: 'American History X (1998)'
Finding best predictor: 'FeardotCom (2002)'
Finding best predictor: 'Halloween (1978)'
Finding best predictor: 'Along Came a Spider (2002)'
Finding best predictor: 'The Mist (2007)'
Finding best predictor: 'Aladdin (1992)'
Finding best predictor: 'Pirates of the Caribbean: The Curse of the Black Pearl (2003)'
Finding best pr

In [ ]:
# ^ 
# R^2 - The proportion of the variance in the predicted variable (y) that can be explained by the features (X)
# R^2 = COD coefficient of determination 
# R^2 in (0,1), larger is better

# RMSE 
# A representation of the average distance between the observed data values and the predicted data values
# 0+, lower is better

In [ ]:
print(len(best_data_all))

In [52]:
# best, worst, avg_cod = rank_scores(this_movie_scores)

# print(best)
# print(worst)
# print(avg_cod)

In [53]:
# best_CODs

In [54]:
print(Average(best_CODs))

0.42378171067196


In [55]:
# now, find the top ten CODs and which movies they are predicted for, and their COD

In [56]:
easiest = sorted(best_CODs, reverse=True)[:10]
easiest

[0.731507476731657,
 0.7315074767316568,
 0.7135542589926913,
 0.7135542589926913,
 0.7112222468014324,
 0.7112222468014324,
 0.7005689836445022,
 0.7005689836445022,
 0.7001881161214467,
 0.6927335239652475]

In [57]:
hardest = sorted(best_CODs)[:10]
hardest

[0.07948469093084642,
 0.11134259626426424,
 0.11708033979272647,
 0.141426437225317,
 0.1438868695548512,
 0.14851372649350136,
 0.15413567330482103,
 0.16016372820860814,
 0.168991422823908,
 0.17111918539600846]

In [58]:
# get the best predictors
print("best predictors:")
best_well_predicted = []
for i in range(len(best_data_all)):
    
    item = best_data_all[i]
    if item['cod'] in easiest:
        print(item)
        best_well_predicted.append(item)
        

best predictors:
{'movie': 'Erik the Viking (1989)', 'other_movie': 'I.Q. (1994)', 'cod': 0.7315074767316568}
{'movie': 'I.Q. (1994)', 'other_movie': 'Erik the Viking (1989)', 'cod': 0.731507476731657}
{'movie': 'The Bandit (1996)', 'other_movie': 'Best Laid Plans (1999)', 'cod': 0.7112222468014324}
{'movie': 'Best Laid Plans (1999)', 'other_movie': 'The Bandit (1996)', 'cod': 0.7112222468014324}
{'movie': 'Congo (1995)', 'other_movie': 'The Straight Story (1999)', 'cod': 0.7005689836445022}
{'movie': 'The Straight Story (1999)', 'other_movie': 'Congo (1995)', 'cod': 0.7005689836445022}
{'movie': 'Ran (1985)', 'other_movie': 'Heavy Traffic (1973)', 'cod': 0.6927335239652475}
{'movie': 'Heavy Traffic (1973)', 'other_movie': 'Ran (1985)', 'cod': 0.6927335239652475}
{'movie': 'The Final Conflict (1981)', 'other_movie': 'The Lookout (2007)', 'cod': 0.7001881161214467}
{'movie': 'The Lookout (2007)', 'other_movie': 'Patton (1970)', 'cod': 0.7135542589926913}
{'movie': 'Patton (1970)', 'othe

In [59]:
# get the worst predictors
print("worst predictors:")
least_well_predicted = []
for i in range(len(best_data_all)):
    
    item = best_data_all[i]
    if item['cod'] in hardest:
        print(item)
        least_well_predicted.append(item)

worst predictors:
{'movie': 'Black Swan (2010)', 'other_movie': 'Sorority Boys (2002)', 'cod': 0.11708033979272647}
{'movie': 'The Fast and the Furious (2001)', 'other_movie': 'Terminator 3: Rise of the Machines (2003)', 'cod': 0.168991422823908}
{'movie': '13 Going on 30 (2004)', 'other_movie': "Can't Hardly Wait (1998)", 'cod': 0.16016372820860814}
{'movie': 'Clueless (1995)', 'other_movie': 'Escape from LA (1996)', 'cod': 0.141426437225317}
{'movie': 'Avatar (2009)', 'other_movie': 'Bad Boys (1995)', 'cod': 0.07948469093084642}
{'movie': 'Interstellar (2014)', 'other_movie': 'Torque (2004)', 'cod': 0.11134259626426424}
{'movie': 'The Cabin in the Woods (2012)', 'other_movie': 'The Evil Dead (1981)', 'cod': 0.1438868695548512}
{'movie': 'Grown Ups 2 (2013)', 'other_movie': 'The Core (2003)', 'cod': 0.17111918539600846}
{'movie': 'Titanic (1997)', 'other_movie': 'Cocktail (1988)', 'cod': 0.15413567330482103}
{'movie': 'La La Land (2016)', 'other_movie': 'The Lookout (2007)', 'cod': 0.

## REGULARIZED REGRESSION - Ridge


In [45]:
# on to Question 3 about regularized regression
# the mean is usually the y intercept.
# if you know nothing about your dataset, the mean is the best guess of prediction



In [46]:
# pick 10 other movies

# Pick 30 movies to build my model
# df_sample = df.sample(n=10,axis='columns')
# df_sample.columns

In [66]:
# 10 other movies, picked randomly or chosen by me:

regularization_movies = [
    'Spirited Away (2001)', #high
    'Inception (2010)',     #high
    'Star Wars: Episode VI - The Return of the Jedi (1983)', # high ish
    'Brazil (1985)',     #selected by me
    'A Night at the Roxbury (1998)', # selected by me
    'City of God (2002)',
    'Indiana Jones and the Temple of Doom (1984)',
    'Mulholland Dr. (2001)',
    'Just Like Heaven (2005)',
    'Black Hawk Down (2001)'
]



In [67]:
# pick 30 movies in the middle of COD
# median = len(best_cods)
# middle = sorted(best_CODs, reverse=True)[:10]

best_CODs.sort()
mid = len(best_CODs) // 2
res = (best_CODs[mid] + best_CODs[~mid]) / 2
start = mid - 15
end = mid + 15
middle = best_CODs[start:end]
middle
print(len(middle))


30


In [68]:
# get the middle movies
print("best predictors:")
middle_predicted = []
middle_predicted_movie_names = []
for i in range(len(best_data_all)):
    
    item = best_data_all[i]
    if item['cod'] in middle:
        print(item)
        middle_predicted.append(item)
        middle_predicted_movie_names.append(item['movie'])

best predictors:
{'movie': 'Let the Right One In (2008)', 'other_movie': 'Slackers (2002)', 'cod': 0.440634409724456}
{'movie': 'The Machinist (2004)', 'other_movie': 'Escape from LA (1996)', 'cod': 0.428765958035613}
{'movie': 'Man on Fire (2004)', 'other_movie': 'Cool Hand Luke (1967)', 'cod': 0.44722999341040515}
{'movie': 'Crossroads (2002)', 'other_movie': 'Pieces of April (2003)', 'cod': 0.41250649808259143}
{'movie': 'The Poseidon Adventure (1972)', 'other_movie': 'The King of Marvin Gardens (1972)', 'cod': 0.4387589579009237}
{'movie': 'The Rock (1996)', 'other_movie': 'The King of Marvin Gardens (1972)', 'cod': 0.43839183751048394}
{'movie': 'Gone in Sixty Seconds (2000)', 'other_movie': 'The 51st State (2001)', 'cod': 0.41102463304683523}
{'movie': 'Blues Brothers 2000 (1998)', 'other_movie': 'The 51st State (2001)', 'cod': 0.4216782524022984}
{'movie': 'Equilibrium (2002)', 'other_movie': 'Change of Habit (1969)', 'cod': 0.4433325904344152}
{'movie': 'The Blue Lagoon (1980)'

In [69]:
# so, we have our 30 movies in the middle, and the other 10 movies randomly picked

In [70]:
randomly_chosen_movies_df = df[regularization_movies]
randomly_chosen_movies_df

,Spirited Away (2001),Inception (2010),Star Wars: Episode VI - The Return of the Jedi (1983),Brazil (1985),A Night at the Roxbury (1998),City of God (2002),Indiana Jones and the Temple of Doom (1984),Mulholland Dr. (2001),Just Like Heaven (2005),Black Hawk Down (2001)
0,3.078532,3.0,2.886329,2.560989,2.602381,2.791783,4.000000,2.757022,2.585244,2.737605
1,3.070740,2.5,2.878537,2.553197,2.594589,2.783990,2.726422,2.749230,2.577452,2.729813
2,4.000000,4.0,3.172308,2.846968,2.888360,3.077761,3.020193,3.043001,2.871223,3.023584
3,2.914420,3.5,4.000000,2.396878,2.438269,2.627671,2.500000,2.592910,2.421133,2.573494
4,2.840578,3.5,2.648375,2.323035,2.364427,2.553829,0.500000,2.519068,2.347290,2.499651
...,...,...,...,...,...,...,...,...,...,...
1092,3.307103,4.0,4.000000,2.789560,2.830952,3.020354,3.500000,2.985593,2.813816,2.966176
1093,4.000000,3.0,4.000000,3.500000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
1094,3.273368,3.0,4.000000,2.755825,2.797217,2.986619,2.929051,2.951858,2.780081,2.932442
1095,3.402416,2.5,4.000000,2.884873,2.926265,3.115666,3.058098,3.080906,2.909128,3.061489


In [71]:
# next we just pick one of the 30 middle movies and try the ridge regression 

ridge_movie = 'Let the Right One In (2008)'
ridge_df = df[ridge_movie] 

ridge_all_df = randomly_chosen_movies_df.copy()
ridge_all_df[ridge_movie] = ridge_df

In [72]:
ridge_all_df

,Spirited Away (2001),Inception (2010),Star Wars: Episode VI - The Return of the Jedi (1983),Brazil (1985),A Night at the Roxbury (1998),City of God (2002),Indiana Jones and the Temple of Doom (1984),Mulholland Dr. (2001),Just Like Heaven (2005),Black Hawk Down (2001),Let the Right One In (2008)
0,3.078532,3.0,2.886329,2.560989,2.602381,2.791783,4.000000,2.757022,2.585244,2.737605,2.619604
1,3.070740,2.5,2.878537,2.553197,2.594589,2.783990,2.726422,2.749230,2.577452,2.729813,2.611812
2,4.000000,4.0,3.172308,2.846968,2.888360,3.077761,3.020193,3.043001,2.871223,3.023584,2.905583
3,2.914420,3.5,4.000000,2.396878,2.438269,2.627671,2.500000,2.592910,2.421133,2.573494,2.455492
4,2.840578,3.5,2.648375,2.323035,2.364427,2.553829,0.500000,2.519068,2.347290,2.499651,2.381650
...,...,...,...,...,...,...,...,...,...,...,...
1092,3.307103,4.0,4.000000,2.789560,2.830952,3.020354,3.500000,2.985593,2.813816,2.966176,2.848175
1093,4.000000,3.0,4.000000,3.500000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.500000
1094,3.273368,3.0,4.000000,2.755825,2.797217,2.986619,2.929051,2.951858,2.780081,2.932442,2.814440
1095,3.402416,2.5,4.000000,2.884873,2.926265,3.115666,3.058098,3.080906,2.909128,3.061489,2.943488


In [73]:
# now what? our goal with regularization is to build a simple model which captures the important features
# balance between simple and not overfitting

In [74]:
X = randomly_chosen_movies_df
Y = ridge_df

In [75]:
x = randomly_chosen_movies_df.values
y = ridge_df.values

x.size, y.size

(10960, 1096)

# cross validation splitter
We divide our data set into K-folds. K represents the number of folds into which you want to split your data. If we use 5-folds, the data set divides into five sections. In different iterations, one part becomes the validation set.

In [ ]:
# specify number of folds for 80/20 split each iteration
cv = 5

alphas = np.arange(10.1, 50.1, 0.0025)
scoring = 'neg_root_mean_squared_error' # is this true?

In [ ]:
# the optimal lambda comes from cross validation! we need to split into train versus test to get optimal lambda

In [ ]:
# Create the random grid
# param_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

parameters = {'alpha':alphas}

In [ ]:
#$ most people mean center their data anyways so that the beta naught is going to be zero
# otherwise you havbe to worry about sklearn including the beta naught in tghe ridge regression.
# lab uses gridsearchCV to find the optimal hyperparams

# fyi we are attempting to minimize RMSE

# ridge_cv = RidgeCV() # built in cross validation
model = Ridge()
# ridge_reg = GridSearchCV(estimator = random_forest, param_grid = param_grid, scoring="accuracy")


model.get_params().keys()


In [80]:
# define the grid search
# ridge_reg = GridSearchCV(ridge_cv, param_grid=parameters, scoring='neg_root_mean_squared_error', cv=cv)
ridge_reg = GridSearchCV(model, param_grid=parameters, scoring='neg_root_mean_squared_error', cv=cv)

In [84]:
#fit the grid search

# from lab
# rf_grid.fit(X_train, y_train)

# from online
# ridge_reg.fit(X,y)
ridge_reg.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [83]:
# best estimator
print(ridge_reg.best_estimator_)



Ridge(alpha=31.7049999999957)


In [ ]:
# best model
best_model = ridge_reg.best_estimator_
best_model.fit(X,y)

In [120]:
# ridge_reg.cv_results_

In [106]:
alphas=np.arange(0.1, 80.1, 0.025)
alphas

array([ 0.1  ,  0.125,  0.15 , ..., 80.025, 80.05 , 80.075])

In [115]:
# lets also try it a different way
# fit it on the training dataset
model2 = RidgeCV(alphas=alphas, cv=cv, scoring='neg_root_mean_squared_error')

In [139]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2) # Divides data into train and test splits given the size of one of them
X_train.shape, X_test.shape

((877, 10), (220, 10))

In [117]:
# fit model
model2.fit(X_train, y_train)
# summarize chosen configuration
print('alpha: %f' % model2.alpha_)

alpha: 29.275000


In [118]:
# fit model
model2.fit(X, Y)
# summarize chosen configuration
print('alpha: %f' % model2.alpha_)

alpha: 27.425000


In [ ]:
# alphas=np.arange(0.1, 80.1, 0.025) 
# 29.275, 27.425
# 23.0, 27.425
# 17.9, 27.425



In [143]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2) # Divides data into train and test splits given the size of one of them
X_train.shape, X_test.shape

((877, 10), (220, 10))

In [144]:
#different each time if i split before fitting
# try without ridgeccv , just plain ridge
parameters = {'alpha':alphas}


model3 = Ridge()
ridge_reg3 = GridSearchCV(model3, param_grid=parameters, scoring='neg_root_mean_squared_error', cv=cv)

In [152]:
# fit model
ridge_reg3.fit(X_train, y_train)
# summarize chosen configuration
# print('alpha: %f' % model3.alpha_)

# best estimator
print(ridge_reg3.best_estimator_)

Ridge(alpha=31.824999999999996)


In [142]:
# fit model
ridge_reg3.fit(X, Y)
# summarize chosen configuration
# print('alpha: %f' % model3.alpha_)

# best estimator
print(ridge_reg3.best_estimator_)

Ridge(alpha=27.424999999999994)


In [164]:
print(ridge_reg3.best_estimator_.alpha)

30.024999999999995


In [165]:
print(ridge_reg3.best_estimator_)

Ridge(alpha=30.024999999999995)


In [ ]:
# alphas=np.arange(0.1, 80.1, 0.025) # re-executed test train split
# 29.274, 27.424
# 39.024, 27.425
# 31.824, 27.424



In [137]:
alphas

array([ 0.1  ,  0.125,  0.15 , ..., 80.025, 80.05 , 80.075])

In [ ]:
# rr = Ridge(alpha=0.01)
# rr.fit(X_train, y_train) 

# pred_train_rr= rr.predict(X_train)
# print(np.sqrt(mean_squared_error(y_train,pred_train_rr)))
# print(r2_score(y_train, pred_train_rr))

# pred_test_rr= rr.predict(X_test)
# print(np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
# print(r2_score(y_test, pred_test_rr))

In [151]:
# online example
rr = Ridge(alpha=27.424)
rr.fit(X_train, y_train) 

# pred_train_rr= rr.predict(X_train)
# print("train:")
# print(np.sqrt(mean_squared_error(y_train,pred_train_rr)))
# print(r2_score(y_train, pred_train_rr))

print("test:")
pred_test_rr= rr.predict(X_test)
print("RMSE: %s" % np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R^2:  %s" % r2_score(y_test, pred_test_rr))

test:
RMSE: 0.2945457508436924
R^2: 0.512771215086242


In [ ]:
# from the lab (before tuning)
# model = PolynomialRidgeRegression(30)
# model.fit(X, Y)
# y_test = model.predict(X_test)

# plt.figure(figsize=(12, 10))
# plt.scatter(X.ravel(), Y)
# plt.plot(X_test.ravel(), y_test)
# plt.title("mean squared error for Ridge Regularization: {0:.3g}".format(mean_squared_error(model.predict(X), Y)))
# plt.ylim(-4, 14)
# plt.xlabel("x1")
# plt.ylabel("x2")
# plt.show()
# print("Model Coef: ", model[1].coef_)

In [153]:
mod = Ridge(alpha=27.424)
mod.fit(X, Y)

y_test = mod.predict(X_test)

In [157]:
# x_r = X.values

# plt.figure(figsize=(12, 10))
# # plt.scatter(X.ravel(), Y)
# plt.scatter(x_r.ravel(), Y)
# plt.plot(X_test.ravel(), y_test)
# plt.title("mean squared error for Ridge Regularization: {0:.3g}".format(mean_squared_error(mod.predict(X), Y)))
# plt.ylim(-4, 14)
# plt.xlabel("x1")
# plt.ylabel("x2")
# plt.show()
print("Model Coef: ", mod.coef_)

Model Coef:  [0.02730979 0.00288969 0.05538064 0.28413765 0.02769813 0.1068413
 0.02776218 0.09639236 0.14923608 0.05253502]


In [ ]:
# okay, now i need a function which will take in the movie (1/30)
# and find the optimal alpha
# then use that alpha in building a model
# fit the model to all the data
# predict on test set, measure error on test set

In [166]:
# function to get the best alpha for a train set

alphas=np.arange(0.1, 80.1, 0.025) 

def get_best_alpha(X_train, y_train):
    
    parameters = {'alpha':alphas}
    model = Ridge()
    ridge_reg = GridSearchCV(model, param_grid=parameters, scoring='neg_root_mean_squared_error', cv=cv)

    ridge_reg3.fit(X_train, y_train)
    best_alpha = ridge_reg3.best_estimator_.alpha
    
    return best_alpha

In [172]:
# for movie in picked_movies:

X = randomly_chosen_movies_df # the dataframe containing my 10 predictor movies 

# picked_movies = ['Let the Right One In (2008)']
for movie in middle_predicted_movie_names:
    
    print("movie: %s" % movie)

    # get the data, split it
    Y = df[movie]   
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2) # Divides data into train and test splits given the size of one of them

    # get the best alpha
    best_alpha = get_best_alpha(X_train, y_train)
    print("\talpha: %s" % best_alpha)
    
    # build a new model using this alpha
    mod = Ridge(alpha=best_alpha)
    mod.fit(X, Y)

    y_test = mod.predict(X_test) 
    # SUMANYU calculated this as -->  mean_squared_error(model.predict(X), Y))
    print("\tRMSE test: %s" % np.sqrt(mean_squared_error(y_test,pred_test_rr)))
    print("\tRMSE all : %s" % np.sqrt(mean_squared_error(mod.predict(X), Y)))
    print("\tModel Coef: ", mod.coef_)
    
    print("")
    

movie: Let the Right One In (2008)
	alpha: 28.574999999999996
	RMSE test: 0.32765516730206085
	RMSE all : 0.2902682307082177
	Model Coef:  [0.02748225 0.00294063 0.05547026 0.2820795  0.02833694 0.1065029
 0.02793606 0.0963477  0.14908164 0.05276658]

movie: The Machinist (2004)
	alpha: 47.44999999999999
	RMSE test: 0.3729854008270341
	RMSE all : 0.3230216756487641
	Model Coef:  [0.04134637 0.0509918  0.03727091 0.23135544 0.08023391 0.09900573
 0.02504793 0.15302937 0.05440043 0.06415711]

movie: The Godfather: Part II (1974)
	alpha: 67.09999999999998
	RMSE test: 0.47557140343033905
	RMSE all : 0.46002670870628726
	Model Coef:  [0.006114   0.07178625 0.08119217 0.13862939 0.04414046 0.0564038
 0.04203366 0.09211196 0.10477619 0.16689679]

movie: Showgirls (1995)
	alpha: 22.074999999999996
	RMSE test: 0.39716820196937175
	RMSE all : 0.30931279092336417
	Model Coef:  [-0.01341827 -0.02698885  0.02670294  0.15292453  0.2188831   0.06011498
  0.02045822  0.06313532  0.25500251  0.08957345